# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

# 1. Загрузка и подготовка данных

In [2]:
data_0 = pd.read_csv('datasets/geo_data_0.csv')
data_1 = pd.read_csv('datasets/geo_data_1.csv')
data_2 = pd.read_csv('datasets/geo_data_2.csv')

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
features_0 = data_0[['f0','f1','f2']]
features_1 = data_1[['f0','f1','f2']]
features_2 = data_2[['f0','f1','f2']]
target_0 = data_0['product']
target_1 = data_1['product']
target_2 = data_2['product']

Разбил данные на критерии и уеливые признаки, столбец id удалил, так как он не несет влияние на то сколько нефти в скважине.

# 2. Обучение и проверка модели

In [10]:
features_0_train, features_0_valid, target_0_train, target_0_valid= train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

In [11]:
features_1_train, features_1_valid, target_1_train, target_1_valid= train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

In [12]:
features_2_train, features_2_valid, target_2_train, target_2_valid= train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

Разбил данные на обучающие и валидационные выборки.

### Модель для первого региона

In [13]:
model_region_0 = LinearRegression()
model_region_0.fit(features_0_train,target_0_train)
predicted_region_0 =  pd.Series(model_region_0.predict(features_0_valid))

In [14]:
print('среднее значение по предсказаниям модели:',predicted_region_0.mean())
print('фактическое среднее значение:',target_0.mean())

среднее значение по предсказаниям модели: 92.59256778438038
фактическое среднее значение: 92.50000000000001


In [15]:
print('RMSE модели:',mean_squared_error(predicted_region_0,target_0_valid)**(1/2))

RMSE модели: 37.5794217150813


### Модель для второго региона

In [16]:
model_region_1 = LinearRegression()
model_region_1.fit(features_1_train,target_1_train)
predicted_region_1 = pd.Series(model_region_1.predict(features_1_valid))

In [17]:
print('среднее значение по предсказаниям модели:',predicted_region_1.mean())
print('фактическое среднее значение:',target_1.mean())

среднее значение по предсказаниям модели: 68.72854689544602
фактическое среднее значение: 68.82500000000002


In [18]:
print('RMSE модели:',mean_squared_error(predicted_region_1,target_1_valid)**(1/2))

RMSE модели: 0.8930992867756155


### Модель для третьего региона

In [19]:
model_region_2 = LinearRegression()
model_region_2.fit(features_2_train,target_2_train)
predicted_region_2 =  pd.Series(model_region_2.predict(features_2_valid))

In [20]:
print('среднее значение по предсказаниям модели:',predicted_region_2.mean())
print('фактическое среднее значение:',target_2.mean())

среднее значение по предсказаниям модели: 94.96504596800489
фактическое среднее значение: 95.00000000000004


In [21]:
print('RMSE модели:',mean_squared_error(predicted_region_2,target_2_valid)**(1/2))

RMSE модели: 40.02970873393434


# 3. Подготовка к расчёту прибыли

In [22]:
full_predicted_region_0 =  pd.Series(model_region_0.predict(features_0))
full_predicted_region_2 =  pd.Series(model_region_2.predict(features_2))
full_predicted_region_1 =  pd.Series(model_region_1.predict(features_1))

In [23]:
budget = 10000000000
price = 450000
borehole_count = 200
product = budget/price/borehole_count
product_price = budget/borehole_count
price_by_borehole = budget/borehole_count

Количество нефти в скважине чтобы он была не убыточной, должна быть выше средней по региону, 3й регион ближе всего к этому значению, второй дальше всего

In [24]:
def get_profit(target, predictions):
    predictions = predictions.sort_values(ascending=False)
    selected_boreholes = target[predictions.index][:borehole_count]
    product = selected_boreholes.sum()
    return product * price - budget

In [25]:
print('максимальная прибыль с первого  региона по предсказанию модели:', get_profit(target_0,target_0))
print('максимальная прибыль со второго региона по предсказанию модели:', get_profit(target_1,target_1))
print('максимальная прибыль с третьего региона по предсказанию модели:', get_profit(target_2,target_2))

максимальная прибыль с первого  региона по предсказанию модели: 6635036568.082407
максимальная прибыль со второго региона по предсказанию модели: 2415086696.681511
максимальная прибыль с третьего региона по предсказанию модели: 7059632928.358982


# 4. Расчёт прибыли и рисков 

In [26]:
sample_size = 500
samples_count = 1000

In [27]:
def get_stats(targets_list, predictions_list):
    state = RandomState(12345)
    profit = []
    for i in range(samples_count):
        target_sample = targets_list.sample(sample_size, replace=True, random_state=state)
        predictions_sample = predictions_list[target_sample.index]
        profit.append(get_profit(target_sample, predictions_sample))
    profit = pd.Series(profit)
    mean_profit = profit.mean()
    confidence_interval = (profit.quantile(0.025), profit.quantile(0.975))
    risk = (profit < 0).mean()* 100
    print('среднее по региону:', mean_profit)
    print('доверительный интервал региона:', confidence_interval)
    print('риски региона:', risk)

### Первый регион

In [28]:
get_stats(target_0, full_predicted_region_0)

среднее по региону: 431092313.49565756
доверительный интервал региона: (-70541767.29685411, 979019659.0485121)
риски региона: 5.1


### Второй регион

In [29]:
get_stats(target_1, full_predicted_region_1)

среднее по региону: 462956626.84659
доверительный интервал региона: (55988529.93353825, 845839331.1684114)
риски региона: 1.3


### Третий регион

In [30]:
get_stats(target_2, full_predicted_region_2)

среднее по региону: 380554488.03086865
доверительный интервал региона: (-177060947.06485948, 896985478.3182544)
риски региона: 9.1


## Вывод
Второй регион наиболее перспективный для развертывания скважин. В нем наибольшая средняя прибыль со скважен, доверительный интервал показывает хорошую прибыль и не уходит в отрицательные значения, а риски минимальны: 1.3%